# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data1= data[['CustomerID','ProductName','Quantity']]

data2 = data1.groupby(['CustomerID','ProductName']).sum()

data2.head()


Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
#data2['Quantity'] = data2['Quantity'].fillna(0)
table = pd.pivot_table(data2, values='Quantity', index=['ProductName'],
             columns=['CustomerID'], fill_value=0)
table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = distances[477].sort_values(ascending=False)[1:]
top5=similarities.head(5)
top5

CustomerID
3535    0.083651
3317    0.081210
1072    0.079792
1008    0.079246
639     0.078977
Name: 477, dtype: float64

In [8]:
top5.index

Int64Index([3535, 3317, 1072, 1008, 639], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
top5_index = list(top5.index)

top5_index

[3535, 3317, 1072, 1008, 639]

In [33]:
data5=data.set_index(['CustomerID'])
data5.head()


,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,
61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [20]:
data6=data5.loc[data5.index.isin(top5_index)]
data6.head()

,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,
1008,Leticia,Krueger,4468566,270,Cheese Cloth No 100,1
3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1
3535,Lana,Peters,2575012,159,Eggplant - Asian,1
3317,Chandra,Benitez,6534614,69,Raspberries - Fresh,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [42]:
data7=data6.groupby(['ProductName']).sum()

data7=data7.sort_values(by='Quantity',ascending=False)
data7=data7.drop(['SalesID','ProductID'], axis=1)
data7.head(10)

,Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3
Soup - Campbells Bean Medley,3
"Sauce - Gravy, Au Jus, Mix",3
Cheese - Cambozola,3
Soupfoamcont12oz 112con,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [68]:
merged=data7.merge(table, on='ProductName')
# My chosen customer was ID= 477
unpurchased=merged.loc[merged[477]==0].head(5)
unpurchased

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Olive - Spread Tapenade,3,0,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Sprouts - Baby Pea Tendrils,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,25,0
Wine - Blue Nun Qualitatswein,3,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
"Veal - Inside, Choice",3,2,0,0,0,1,0,0,1,0,...,0,0,0,0,25,0,0,0,0,0
"Pepper - Black, Whole",3,1,1,0,0,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [70]:
unpurchased5=list(unpurchased.index)
unpurchased5

['Olive - Spread Tapenade',
 'Sprouts - Baby Pea Tendrils',
 'Wine - Blue Nun Qualitatswein',
 'Veal - Inside, Choice',
 'Pepper - Black, Whole']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [76]:
dict1={}
customer_ids= set(data['CustomerID'])
customer_ids=list(customer_ids) #Unique list of all Customer IDs

for i in customer_ids:
    similarities = distances[i].sort_values(ascending=False)[1:]
    top5=similarities.head(5)
    top5_index = list(top5.index)
    data6=data5.loc[data5.index.isin(top5_index)]
    data7=data6.groupby(['ProductName']).sum()
    data7=data7.sort_values(by='Quantity',ascending=False)
    data7=data7.drop(['SalesID','ProductID'], axis=1)
    merged=data7.merge(table, on='ProductName')
    unpurchased=merged.loc[merged[i]==0].head(5)
    unpurchased5=list(unpurchased.index)
    dict1[i]=unpurchased5
    
dict1 #No lo voy a imprimir que sale largisimo todo el diccionario! (pero funciona bien jejeje)

    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [84]:
df=pd.DataFrame(dict1).T
df.columns=['Recommendation 1','Recommendation 2','Recommendation 3','Recommendation 4','Recommendation 5']
df.head()

,Recommendation 1,Recommendation 2,Recommendation 3,Recommendation 4,Recommendation 5
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [85]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)
dict1={}
customer_ids= set(data['CustomerID'])
customer_ids=list(customer_ids) #Unique list of all Customer IDs

for i in customer_ids:
    similarities = distances[i].sort_values(ascending=False)[1:]
    top5=similarities.head(5)
    top5_index = list(top5.index)
    data6=data5.loc[data5.index.isin(top5_index)]
    data7=data6.groupby(['ProductName']).sum()
    data7=data7.sort_values(by='Quantity',ascending=False)
    data7=data7.drop(['SalesID','ProductID'], axis=1)
    merged=data7.merge(table, on='ProductName')
    unpurchased=merged.loc[merged[i]==0].head(5)
    unpurchased5=list(unpurchased.index)
    dict1[i]=unpurchased5
    df=pd.DataFrame(dict1).T
    df.columns=['Recommendation 1','Recommendation 2','Recommendation 3','Recommendation 4','Recommendation 5']

df.head()


,Recommendation 1,Recommendation 2,Recommendation 3,Recommendation 4,Recommendation 5
83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
59399,Anchovy Paste - 56 G Tube,Ezy Change Mophandle,Garlic - Peeled,Beans - Wax,Broom - Corn
92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round
49159,"Turnip - White, Organic",Peas - Frozen,Lamb - Ground,Beer - Blue,Beef - Ground Medium
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%


In [86]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cityblock'))), 
                         index=table.columns, columns=table.columns)
dict1={}
customer_ids= set(data['CustomerID'])
customer_ids=list(customer_ids) #Unique list of all Customer IDs

for i in customer_ids:
    similarities = distances[i].sort_values(ascending=False)[1:]
    top5=similarities.head(5)
    top5_index = list(top5.index)
    data6=data5.loc[data5.index.isin(top5_index)]
    data7=data6.groupby(['ProductName']).sum()
    data7=data7.sort_values(by='Quantity',ascending=False)
    data7=data7.drop(['SalesID','ProductID'], axis=1)
    merged=data7.merge(table, on='ProductName')
    unpurchased=merged.loc[merged[i]==0].head(5)
    unpurchased5=list(unpurchased.index)
    dict1[i]=unpurchased5
    df=pd.DataFrame(dict1).T
    df.columns=['Recommendation 1','Recommendation 2','Recommendation 3','Recommendation 4','Recommendation 5']

df.head()

,Recommendation 1,Recommendation 2,Recommendation 3,Recommendation 4,Recommendation 5
83973,"Pork - Bacon, Double Smoked","Nut - Chestnuts, Whole",Olive - Spread Tapenade,Butter - Unsalted,Beef Ground Medium
59399,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pork - Bacon, Double Smoked",Soup - Campbells Bean Medley,"Salsify, Organic"
92168,Wine - Blue Nun Qualitatswein,Muffin Batt - Blueberry Passion,Knife Plastic - White,Chinese Foods - Chicken,"Wine - Magnotta, Merlot Sr Vqa"
49159,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Tia Maria,Veal - Sweetbread,Wine - Chardonnay South
18441,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste",Butter - Unsalted
